In [1]:
import logging

logging.getLogger("sagemaker").setLevel(logging.WARNING)
logging.getLogger("boto3").setLevel(logging.WARNING)

import boto3
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import (
    ParameterString,
    ParameterFloat,
    ParameterInteger,
)
from sagemaker.workflow.steps import ProcessingStep, TrainingStep, CacheConfig
from sagemaker.sklearn.processing import SKLearnProcessor

C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\pydantic\_internal\_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/23/25 18:33:53] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=830005;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=391067;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\tochi\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
# SageMaker session and role
sagemaker_session = sagemaker.Session()
pipeline_session = PipelineSession()

account = boto3.client("sts").get_caller_identity().get("Account")
role = f"arn:aws:iam::{account}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"

[02/23/25 18:33:56] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=611971;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=662309;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[02/23/25 18:33:57] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=92289;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=517157;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[02/23/25 18:33:58] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=761931;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=520347;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

In [3]:
# Define pipeline parameters
years_to_filter = ParameterString(name="Historical_Years", default_value="30")
train_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.large"
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="Approved"
)
# cache_config = CacheConfig(
#     enable_caching=True, expire_after="1d"
# )  # Cache configuration

## Data Ingestion

In [4]:
data_ingestion_processor = SKLearnProcessor(
    framework_version="1.0-1",
    command=["python3"],
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
)

# Data ingestion step
step_data_ingestion = ProcessingStep(
    name="DataIngestion",
    processor=data_ingestion_processor,
    inputs=[],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="snp500",
            source="/opt/ml/processing/output",
            destination="s3://aws-portfolio-projects/snp500-data/input_data/",
        ),
    ],
    code="training_scripts/data_ingestion.py",
    # cache_config=cache_config,
    job_arguments=["--years-to-filter", years_to_filter],
)

## Data Processing Pipeline

In [5]:
data_preprocessor = SKLearnProcessor(
    framework_version="1.0-1", role=role, instance_type="ml.m5.large", instance_count=1
)

# Data processing step
step_data_processing = ProcessingStep(
    name="DataPreprocessing",
    processor=data_preprocessor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            source=step_data_ingestion.properties.ProcessingOutputConfig.Outputs[
                "snp500"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/input",
        )
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="train_data",
            source="/opt/ml/processing/output/train",
            destination="s3://aws-portfolio-projects/snp500-data/train_data/",
        )
    ],
    code="training_scripts/data_processing.py",
    # cache_config=cache_config,
    job_arguments=[
        "--input_path",
        "/opt/ml/processing/input/snp500.csv",
        "--output_dir",
        "/opt/ml/processing/output/train",
    ],
)

## Model training pipeline

In [6]:
from sagemaker.xgboost.estimator import XGBoost

xgboost_estimator = XGBoost(
    entry_point="training_scripts/train.py",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.5-1",
    py_version="py3",
    output_path="s3://aws-portfolio-projects/snp500-data/model_artifacts/",
    base_job_name="xgboost-stockmarket-training-job",
    disable_profiler=True,
)

# Model training step
step_train = TrainingStep(
    name="ModelTraining",
    estimator=xgboost_estimator,
    inputs={
        "train": sagemaker.inputs.TrainingInput(
            s3_data=step_data_processing.properties.ProcessingOutputConfig.Outputs[
                "train_data"
            ].S3Output.S3Uri,
            content_type="text/csv",
        )
    },
    # cache_config=cache_config,
)

## Model Evaluation

In [7]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.processing import ScriptProcessor

evaluation_processor = ScriptProcessor(
    image_uri=sagemaker.image_uris.retrieve(
        framework="xgboost", region=sagemaker_session.boto_region_name, version="1.5-1"
    ),
    command=["python3"],
    instance_type="ml.m5.large",
    instance_count=1,
    base_job_name="model-evaluation",
    role=role,
    sagemaker_session=sagemaker_session,
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json"
)

# Evaluation step
step_evaluation = ProcessingStep(
    name="ModelEvaluation",
    processor=evaluation_processor,
    inputs=[
        sagemaker.processing.ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        sagemaker.processing.ProcessingInput(
            source=step_data_processing.properties.ProcessingOutputConfig.Outputs[
                "train_data"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/input",
        ),
    ],
    outputs=[
        sagemaker.processing.ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation",
            destination="s3://aws-portfolio-projects/snp500-data/evaluation_results/",
        )
    ],
    code="training_scripts/evaluate_model.py",
    property_files=[evaluation_report],
    # cache_config=cache_config,
    job_arguments=[
        "--input-path",
        "/opt/ml/processing/input/train.csv",
        "--model-path",
        "/opt/ml/processing/model",
        "--output-path",
        "/opt/ml/processing/evaluation",
    ],
)

## Model Registration

In [8]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.step_collections import RegisterModel

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_evaluation.arguments["ProcessingOutputConfig"]["Outputs"][0][
                "S3Output"
            ]["S3Uri"]
        ),
        content_type="application/json",
    )
)

# Define the RegisterModel step
step_register = RegisterModel(
    name="RegisterModel",
    estimator=xgboost_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="StockPredictionModels",
    approval_status="Approved",
    model_metrics=model_metrics,
)

[02/23/25 18:34:03] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=999064;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=573911;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

## Model Deployment

In [9]:
from sagemaker.model import Model
from sagemaker.xgboost.model import XGBoostPredictor
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.workflow.model_step import ModelStep

# Define the XGBoost model
model = Model(
    image_uri=sagemaker.image_uris.retrieve(
        framework="xgboost",
        region=pipeline_session.boto_region_name,
        version="1.5-1",
        instance_type="ml.m5.large",
    ),
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
    predictor_cls=XGBoostPredictor,
    entry_point="training_scripts/inference.py",
)

# Create the ModelStep
step_create_model = ModelStep(
    name="StockMarketPrediction",
    step_args=model.create(instance_type="ml.m5.large"),
)

C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [10]:
# Custom Lambda Step
import time
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda


lambda_role = f"arn:aws:iam::{account}:role/sagemaker-pipeline-lambda-role"
current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
model_name = "stock-market-model" + current_time
endpoint_config_name = "stock-prediction-endpoint-config-" + current_time
endpoint_name = "stock-market-prediction-endpoint"

function_name = "stock-prediction-model-lambda-step-endpoint-deploy"

# lambda_helper.py file is used to create the Lambda function
func = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="training_scripts/lambda_helper.py",
    handler="lambda_helper.lambda_handler",
)

output_param_1 = LambdaOutput(
    output_name="statusCode", output_type=LambdaOutputTypeEnum.String
)
output_param_2 = LambdaOutput(
    output_name="body", output_type=LambdaOutputTypeEnum.String
)
output_param_3 = LambdaOutput(
    output_name="other_key", output_type=LambdaOutputTypeEnum.String
)

step_deploy_lambda = LambdaStep(
    name="DeployStep",
    lambda_func=func,
    inputs={
        "model_name": step_create_model.properties.ModelName,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

## Deployment Condition

In [11]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)

cond_lte = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step=step_evaluation,
        property_file=evaluation_report,
        json_path="precision",
    ),
    right=0.54,
)

step_conditional_register = ConditionStep(
    name="stockmarket_precision_cond",
    conditions=[cond_lte],
    if_steps=[step_register, step_deploy_lambda],
    else_steps=[],
)

[02/23/25 18:34:06] WARNING  The class JsonGet has been renamed in sagemaker>=2.                 ]8;id=832904;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\deprecations.py\deprecations.py]8;;\:]8;id=403915;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

## Execute the pipeline

In [12]:
# Define the pipeline
pipeline = Pipeline(
    name="StockTrainingPipeline",
    parameters=[years_to_filter, train_instance_type, model_approval_status],
    steps=[
        step_data_ingestion,
        step_data_processing,
        step_train,
        step_evaluation,
        step_conditional_register,
    ],
    sagemaker_session=sagemaker_session,
)

# Submit the pipeline
pipeline.upsert(role_arn=role)
execution = pipeline.start()
execution.wait()

[02/23/25 18:34:07] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=31635;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=48245;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[02/23/25 18:34:09] WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=52393;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=47828;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[02/23/25 18:34:10] WARNING  Popping out 'TrainingJobName' from the pipeline definition by default ]8;id=407051;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=319752;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             since it will be overridden at pipeline execution time. Please                        
                             utilize the PipelineDefinitionConfig to persist this field in the                     
                             pipeline definition if desired.                                                       

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=445120;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=74988;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  No finished training job found associated with this estimator.       ]8;id=231345;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\estimator.py\estimator.py]8;;\:]8;id=336833;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

...

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=717442;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=931563;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  No finished training job found associated with this estimator.       ]8;id=838650;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\estimator.py\estimator.py]8;;\:]8;id=668369;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  Popping out 'CertifyForMarketplace' from the pipeline definition since   ]8;id=677016;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\_utils.py\_utils.py]8;;\:]8;id=390453;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\_utils.py#515\515]8;;\
                             it will be overridden in pipeline execution time.                                     

                    WARNING  Popping out 'ModelPackageName' from the pipeline definition by        ]8;id=222267;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=583017;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

[02/23/25 18:34:13] WARNING  Popping out 'ModelName' from the pipeline definition by default since ]8;id=135244;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py\utilities.py]8;;\:]8;id=20394;file://C:\Users\tochi\anaconda3\envs\sagemaker-dev\lib\site-packages\sagemaker\workflow\utilities.py#465\465]8;;\
                             it will be overridden at pipeline execution time. Please utilize the                  
                             PipelineDefinitionConfig to persist this field in the pipeline                        
                             definition if desired.                                                                

## Schedule Training Pipeline

In [13]:
import boto3
import pytz
from datetime import datetime, timedelta

# Set US Central Time (UTC-6)
central_tz = pytz.timezone("US/Central")

# Set the schedule time to 8 AM US Central Time on the next Sunday
now = datetime.now(central_tz)
next_sunday = now + timedelta((6 - now.weekday()) % 7)
naive_next_sunday = next_sunday.replace(
    hour=8, minute=0, second=0, microsecond=0
).replace(tzinfo=None)
schedule_time = central_tz.localize(naive_next_sunday)

# Convert to UTC
schedule_time_utc = schedule_time.astimezone(pytz.utc)

# Define the cron expression for every Sunday at 8 AM US Central Time
cron_expression = "cron(0 14 ? * 1 *)"  # 8 AM Central Time is 2 PM UTC

# Create EventBridge client
events_client = boto3.client("events")

# Create the EventBridge rule
rule_response = events_client.put_rule(
    Name="SageMakerTrainingPipelineSchedule",
    ScheduleExpression=cron_expression,
    State="ENABLED",
    Description="Trigger SageMaker pipeline every Sunday at 8 AM US Central Time",
)

# Get the rule ARN
rule_arn = rule_response["RuleArn"]

# Define the target to invoke the Lambda function
sagemaker_training_arn = (
    f"arn:aws:sagemaker:us-east-2:{account}:pipeline/StockTrainingPipeline"
)

# Add the target to the rule
events_client.put_targets(
    Rule="SageMakerTrainingPipelineSchedule",
    Targets=[
        {"Id": "SageMakerTrainingPipelineSchedule", "Arn": sagemaker_training_arn}
    ],
)